<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---logging" data-toc-modified-id="Setup---logging-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - logging</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li></ul></li><li><span><a href="#Find-articles-to-be-coded" data-toc-modified-id="Find-articles-to-be-coded-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Find articles to be coded</a></span><ul class="toc-item"><li><span><a href="#which-articles-have-already-been-coded?" data-toc-modified-id="which-articles-have-already-been-coded?-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>which articles have already been coded?</a></span><ul class="toc-item"><li><span><a href="#Tag-the-coded-articles" data-toc-modified-id="Tag-the-coded-articles-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Tag the coded articles</a></span></li><li><span><a href="#Profile-the-coded-articles" data-toc-modified-id="Profile-the-coded-articles-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Profile the coded articles</a></span></li></ul></li><li><span><a href="#tag-all-local-news" data-toc-modified-id="tag-all-local-news-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>tag all local news</a></span><ul class="toc-item"><li><span><a href="#TODO" data-toc-modified-id="TODO-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>TODO</a></span><ul class="toc-item"><li><span><a href="#DONE" data-toc-modified-id="DONE-3.2.1.1"><span class="toc-item-num">3.2.1.1&nbsp;&nbsp;</span>DONE</a></span></li></ul></li><li><span><a href="#Grand-Rapids-Press-local-news" data-toc-modified-id="Grand-Rapids-Press-local-news-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Grand Rapids Press local news</a></span></li><li><span><a href="#Detroit-News-local-news" data-toc-modified-id="Detroit-News-local-news-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Detroit News local news</a></span></li></ul></li></ul></li><li><span><a href="#Code-Articles" data-toc-modified-id="Code-Articles-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Code Articles</a></span><ul class="toc-item"><li><span><a href="#Optional-Validation" data-toc-modified-id="Optional-Validation-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Optional Validation</a></span><ul class="toc-item"><li><span><a href="#Validate-success-publications" data-toc-modified-id="Validate-success-publications-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Validate success publications</a></span></li><li><span><a href="#Validate-error-publications" data-toc-modified-id="Validate-error-publications-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Validate error publications</a></span></li></ul></li></ul></li><li><span><a href="#TODO" data-toc-modified-id="TODO-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>TODO</a></span></li></ul></div>

# Introduction

- Back to [Table of Contents](#Table-of-Contents)

This is a notebook that expands on the OpenCalais code in the file `article_coding.py`, also in this folder.  It includes more sections on selecting publications you want to submit to OpenCalais as an example.  It is intended to be copied and re-used.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
debug_flag = False

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [2]:
import datetime
from django.db.models import Avg, Max, Min
import logging
import six

## Setup - logging

- Back to [Table of Contents](#Table-of-Contents)

configure logging for this notebook's kernel (If you do not run this cell, you'll get the django application's logging configuration.

In [3]:
logging.basicConfig(
    level = logging.INFO,
    format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    filename = '/home/jonathanmorgan/logs/django-research.log',
    filemode = 'w' # set to 'a' if you want to append, rather than overwrite each time.
)

## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [4]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [5]:
%run $django_init_path

django initialized at 2019-08-03 04:37:53.504880


In [6]:
# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.collectors.newsbank.newspapers.DTNB import DTNB
from context_text.models import Article
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase

# Find articles to be coded

- Back to [Table of Contents](#Table-of-Contents)

Tag all locally implemented hard news articles in database and all that have already been coded using Open Calais V2, then work through using OpenCalais to code all local hard news that hasn't alredy been coded, starting with those proximal to the coding sample for methods paper.

## which articles have already been coded?

- Back to [Table of Contents](#Table-of-Contents)

More precisely, find all articles that have Article_Data coded by the automated coder with type "OpenCalais_REST_API_v2" and tag the articles as "coded-open_calais_v2" or something like that.

Then, for articles without that tag, use our criteria for local hard news to filter out and tag publications in the year before and after the month used to evaluate the automated coder, in both the Grand Rapids Press and the Detroit News, so I can look at longer time frames, then code all articles currently in database.

Eventually, then, we'll code and examine before and after layoffs.

In [7]:
# look for publications that have article data:
# - coded by automated coder
# - with coder type of "OpenCalais_REST_API_v2"

# get automated coder
automated_coder_user = ArticleCoder.get_automated_coding_user()

print( "{} - Loaded automated user: {}, id = {}".format( datetime.datetime.now(), automated_coder_user, automated_coder_user.id ) )

2019-08-03 04:37:57.911578 - Loaded automated user: automated, id = 2


In [ ]:
# try aggregates
article_qs = Article.objects.all()
pub_date_info = article_qs.aggregate( Max( 'pub_date' ), Min( 'pub_date' ) )
print( pub_date_info )

In [ ]:
# find articles with Article_Data created by the automated user...
article_qs = Article.objects.filter( article_data__coder = automated_coder_user )

# ...and specifically coded using OpenCalais V2...
article_qs = article_qs.filter( article_data__coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION )

# ...and finally, we just want the distinct articles by ID.
article_qs = article_qs.order_by( "id" ).distinct( "id" )

# count?
article_count = article_qs.count()
print( "Found {} articles".format( article_count ) )

### Tag the coded articles

- Back to [Table of Contents](#Table-of-Contents)

Removing duplicates present from joining with Article_Data yields 579 articles that were coded by the automated coder.

Tag all the coded articles with `OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME`.

In [ ]:
# declare variables
current_article = None
tag_name_list = None
article_count = None
untagged_count = None
already_tagged_count = None
newly_tagged_count = None
count_sum = None
do_add_tag = False

# init
do_add_tag = True

# get article_count
article_count = article_qs.count()

# loop over articles.
untagged_count = 0
already_tagged_count = 0
newly_tagged_count = 0
for current_article in article_qs:
    
    # get list of tags for this publication
    tag_name_list = current_article.tags.names()
    
    # is the coded tag in the list?
    if ( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME not in tag_name_list ):
        
        # are we adding tag?
        if ( do_add_tag == True ):

            # add tag.
            current_article.tags.add( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )
            newly_tagged_count += 1
            
        else:

            # for now, increment untagged count
            untagged_count += 1
            
        #-- END check to see if we are adding tag. --#
        
    else:
        
        # already tagged
        already_tagged_count += 1
        
    #-- END check to see if coded tag is set --#
    
#-- END loop over articles. --#

print( "Article counts:" )
print( "- total articles: {}".format( article_count ) )
print( "- untagged articles: {}".format( untagged_count ) )
print( "- already tagged: {}".format( already_tagged_count ) )
print( "- newly tagged: {}".format( newly_tagged_count ) )
count_sum = untagged_count + already_tagged_count + newly_tagged_count
print( "- count sum: {}".format( count_sum ) )

### Profile the coded articles

- Back to [Table of Contents](#Table-of-Contents)

Look at range of pub dates.

In [ ]:
tags_in_list = []
tags_in_list.append( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )
article_qs = Article.objects.filter( tags__name__in = tags_in_list )
print( "Matching article count: {}".format( article_qs.count() ) )

- Original: 579
- after coding 10: 589 (tag is being set correctly by Open Calais V2 coder)
- 2019.08.02 - after 5000 (minus a few errors because 2 seconds isn't quite enough for rate limit): 5518

In [ ]:
# profile these publications
min_pub_date = None
max_pub_date = None
current_pub_date = None
pub_date_count = None
date_to_count_map = {}
date_to_articles_map = {}
pub_date_article_dict = None

# try aggregates
pub_date_info = article_qs.aggregate( Max( 'pub_date' ), Min( 'pub_date' ) )
print( pub_date_info )

# counts of pubs by date
for current_article in article_qs:
    
    # get pub_date
    current_pub_date = current_article.pub_date
    current_article_id = current_article.id
    
    # get count, increment, and store.
    pub_date_count = date_to_count_map.get( current_pub_date, 0 )
    pub_date_count += 1
    date_to_count_map[ current_pub_date ] = pub_date_count
    
    # also, store up ids and instances
    
    # get dict of article ids to article instances for date
    pub_date_article_dict = date_to_articles_map.get( current_pub_date, {} )
    
    # article already there?
    if ( current_article_id not in pub_date_article_dict ):
        
        # no - add it.
        pub_date_article_dict[ current_article_id ] = current_article
        
    #-- END check to see if article already there.
    
    # put dict back.
    date_to_articles_map[ current_pub_date ] = pub_date_article_dict
    
#-- END loop over articles. --#

# output dates and counts.

# get list of keys from map
keys_list = list( six.viewkeys( date_to_count_map ) )
keys_list.sort()
for current_pub_date in keys_list:
    
    # get count
    pub_date_count = date_to_count_map.get( current_pub_date, 0 )
    print( "- {} ( {} ) count: {}".format( current_pub_date, type( current_pub_date ), pub_date_count ) )
    
#-- END loop over dates --#

In [ ]:
# look at the 2010-07-31 date
pub_date = datetime.datetime.strptime( "2010-07-31", "%Y-%m-%d" ).date()
articles_for_date = date_to_articles_map.get( pub_date, {} )
print( articles_for_date )

# get the article and look at its tags.
article_instance = articles_for_date.get( 6065 )
print( article_instance.tags.all() )

# loop over associated Article_Data instances.
for article_data in article_instance.article_data_set.all():
    
    print( article_data )
    
#-- END loop over associated Article_Data instances --#

## tag all local news

- Back to [Table of Contents](#Table-of-Contents)

Definition of local hard news by in-house implementor for Grand Rapids Press and Detroit News follow.  For each, tag all articles in database that match as "local_hard_news".

### TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- make class for GRPB at NewsBank.

    - also, pull the things that are newspaper specific out of ArticleCoder.py and into the GRPB.py class.

- refine "local news" and "locally created" regular expressions for Grand Rapids Press based on contents of `author_string` and `author_affiliation`.
- do the same for TDN.
- then, use the updated classes and definitions below to flag all local hard news in database for each publication.

#### DONE

- Back to [Table of Contents](#Table-of-Contents)

DONE:

- abstract out shared stuff from GRPB.py and DTNB.py into abstract parent class context_text/collectors/newsbank/newspapers/newsbank_newspaper.py

    - update DTNB.py to use the parent class.
    
- make class for GRPB at NewsBank.

    - context_text/collectors/newsbank/newspapers/GRPB.py

### Grand Rapids Press local news

- Back to [Table of Contents](#Table-of-Contents)

Grand Rapids Press local hard news:

- `context_text/examples/articles/articles-GRP-local_news.py`
- local hard news sections (stored in `Article.GRP_NEWS_SECTION_NAME_LIST`):

    - "Business"
    - "City and Region"
    - "Front Page"
    - "Lakeshore"
    - "Religion"
    - "Special"
    - "State"

- in-house implementor (based on byline patterns, stored in `sourcenet.models.Article.Q_GRP_IN_HOUSE_AUTHOR`):

    - Byline ends in "/ THE GRAND RAPIDS PRESS", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *THE GRAND RAPIDS PRESS$'`

    - Byline ends in "/ PRESS * EDITOR", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *PRESS .* EDITOR$' )`

    - Byline ends in "/ GRAND RAPIDS PRESS * BUREAU", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *GRAND RAPIDS PRESS .* BUREAU$' )`

    - Byline ends in "/ SPECIAL TO THE PRESS", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *SPECIAL TO THE PRESS$' )`
        
- can also exclude columns (I will not):

        grp_article_qs = grp_article_qs.exclude( index_terms__icontains = "Column" )

Need to work to further refine this.

Looking at affiliation strings:

    SELECT author_affiliation, COUNT( author_affiliation ) as affiliation_count
    FROM context_text_article
    WHERE newspaper_id = 1
    GROUP BY author_affiliation
    ORDER BY COUNT( author_affiliation ) DESC;
    
And at author strings for collective bylines:

    SELECT author_string, COUNT( author_string ) as author_count
    FROM context_text_article
    WHERE newspaper_id = 1
    GROUP BY author_string
    ORDER BY COUNT( author_string ) DESC
    LIMIT 10;


In [ ]:
# filter queryset to just locally created Grand Rapids Press (GRP) articles.
# imports
from context_text.models import Article
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase
from context_text.collectors.newsbank.newspapers.GRPB import GRPB

# declare variables - Grand Rapids Press
do_apply_tag = False
tag_to_apply = None
grp_local_news_sections = []
grp_newspaper = None
grp_article_qs = None
article_count = -1

# declare variables - filtering
include_opinion_columns = True
tags_in_list = []
tags_not_in_list = []
filter_out_prelim_tags = False
random_count = -1

# declare variables - make list of article IDs from QS.
article_id_list = []
article_counter = -1
current_article = None
article_tag_name_list = None
article_update_counter = -1

# ==> configure

# configure - size of random sample we want
#random_count = 60

# configure - also, apply tag?
do_apply_tag = True
tag_to_apply = ContextTextBase.TAG_LOCAL_HARD_NEWS

# set up "local, regional and state news" sections
grp_local_news_sections = GRPB.LOCAL_NEWS_SECTION_NAME_LIST

# Grand Rapids Press
# get newspaper instance for GRP.
grp_newspaper = Newspaper.objects.get( id = GRPB.NEWSPAPER_ID )

# start with all articles
#grp_article_qs = Article.objects.all()

# ==> filter to newspaper, local news section list, and in-house reporters.

# ----> manually

# now, need to find local news articles to test on.
#grp_article_qs = grp_article_qs.filter( newspaper = grp_newspaper )

# only the locally implemented sections
#grp_article_qs = grp_article_qs.filter( section__in = grp_local_news_sections )

# and, with an in-house author
#grp_article_qs = grp_article_qs.filter( Article.Q_GRP_IN_HOUSE_AUTHOR )

#print( "manual filter count: {}".format( grp_article_qs.count() ) )

# ----> using Article.filter_articles()
grp_article_qs = Article.filter_articles( qs_IN = grp_article_qs,
                                          newspaper = grp_newspaper,
                                          section_name_list = grp_local_news_sections,
                                          custom_article_q = GRPB.Q_IN_HOUSE_AUTHOR )

print( "Article.filter_articles count: {}".format( grp_article_qs.count() ) )

# and include opinion columns?
if ( include_opinion_columns == False ):
    
    # do not include columns
    grp_article_qs = grp_article_qs.exclude( index_terms__icontains = "Column" )
    
#-- END check to see if we include columns. --#

'''
# filter to newspaper, section list, and in-house reporters.
grp_article_qs = Article.filter_articles( qs_IN = grp_article_qs,
                                          start_date = "2009-12-01",
                                          end_date = "2009-12-31",
                                          newspaper = grp_newspaper,
                                          section_name_list = grp_local_news_sections,
                                          custom_article_q = Article.Q_GRP_IN_HOUSE_AUTHOR )
'''

# how many is that?
article_count = grp_article_qs.count()

print( "Article count before filtering on tags: " + str( article_count ) )

# ==> tags

# tags to exclude
tags_not_in_list = []

# Example: prelim-related tags
#tags_not_in_list.append( "prelim_reliability" )
#tags_not_in_list.append( "prelim_network" ]
#tags_not_in_list.append( "minnesota1-20160328" )
#tags_not_in_list.append( "minnesota2-20160328" )

# for later - exclude articles already coded.
#tags_not_in_list.append( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )

# exclude any already tagged with tag_to_apply
tags_not_in_list.append( tag_to_apply )

if ( ( tags_not_in_list is not None ) and ( len( tags_not_in_list ) > 0 ) ):

    # exclude those in a list
    print( "filtering out articles with tags: " + str( tags_not_in_list ) )
    grp_article_qs = grp_article_qs.exclude( tags__name__in = tags_not_in_list )

#-- END check to see if we have a specific list of tags we want to exclude --#

# include only those with certain tags.
tags_in_list = []

# Examples

# Examples: prelim-related tags
#tags_in_list.append( "prelim_unit_test_001" )
#tags_in_list.append( "prelim_unit_test_002" )
#tags_in_list.append( "prelim_unit_test_003" )
#tags_in_list.append( "prelim_unit_test_004" )
#tags_in_list.append( "prelim_unit_test_005" )
#tags_in_list.append( "prelim_unit_test_006" )
#tags_in_list.append( "prelim_unit_test_007" )

# Example: grp_month
#tags_in_list.append( "grp_month" )

if ( ( tags_in_list is not None ) and ( len( tags_in_list ) > 0 ) ):

    # filter
    print( "filtering to just articles with tags: " + str( tags_in_list ) )
    grp_article_qs = grp_article_qs.filter( tags__name__in = tags_in_list )
    
#-- END check to see if we have a specific list of tags we want to include --#

# filter out "*prelim*" tags?
#filter_out_prelim_tags = True
if ( filter_out_prelim_tags == True ):

    # ifilter out all articles with any tag whose name contains "prelim".
    print( "filtering out articles with tags that contain \"prelim\"" )
    grp_article_qs = grp_article_qs.exclude( tags__name__icontains = "prelim" )
    
#-- END check to see if we filter out "prelim_*" tags --#

# how many is that?
article_count = grp_article_qs.count()

print( "Article count after tag filtering: " + str( article_count ) )

# do we want a random sample?
if ( random_count > 0 ):

    # to get random, order them by "?", then use slicing to retrieve requested
    #     number.
    grp_article_qs = grp_article_qs.order_by( "?" )[ : random_count ]
    
#-- END check to see if we want random sample --#

# this is a nice algorithm, also:
# - http://www.titov.net/2005/09/21/do-not-use-order-by-rand-or-how-to-get-random-rows-from-table/

# make ID list, tag articles if configured to.
article_id_list = []
article_counter = 0
article_update_counter = 0
for current_article in grp_article_qs:

    # increment article_counter
    article_counter += 1

    # add IDs to article_id_list
    article_id_list.append( str( current_article.id ) )
    
    # apply a tag while we are at it?
    if ( ( do_apply_tag == True ) and ( tag_to_apply is not None ) and ( tag_to_apply != "" ) ):
    
        # yes, please.  Tag already present?
        article_tag_name_list = current_article.tags.names()
        if ( tag_to_apply not in article_tag_name_list ):

            # Add tag.
            current_article.tags.add( tag_to_apply )
            
            # increment counter
            article_update_counter += 1
            
        #-- END check to see if tag already present. --#
        
    #-- END check to see if we apply tag. --#

    # output the tags.
    if ( debug_flag == True ):
        print( "- Tags for article " + str( current_article.id ) + " : " + str( current_article.tags.all() ) )
    #-- END DEBUG --#

#-- END loop over articles --#

# output the list.
print( "grp_article_qs count: {}".format( grp_article_qs.count() ) )
print( "Found " + str( article_counter ) + " articles ( " + str( article_count ) + " )." )
print( "- Updated {} articles to add tag {}.".format( article_update_counter, tag_to_apply ) )
if ( debug_flag == True ):
    print( "List of " + str( len( article_id_list ) ) + " local GRP staff article IDs: " + ", ".join( article_id_list ) )
#-- END DEBUG --#


### Detroit News local news

- Back to [Table of Contents](#Table-of-Contents)

Detroit News local news:

- `context_text/examples/articles/articles-TDN-local_news.py`
- local hard news sections (stored in `from context_text.collectors.newsbank.newspapers.DTNB import DTNB` - `DTNB.NEWS_SECTION_NAME_LIST`):

    - "Business"
    - "Metro"
    - "Nation" - because of auto industry stories

- in-house implementor (based on byline patterns, stored in `DTNB.Q_IN_HOUSE_AUTHOR`):

    - Byline ends in "/ The Detroit News", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*the\s*detroit\s*news$' )`

    - Byline ends in "Special to The Detroit News", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*special\s*to\s*the\s*detroit\s*news$' )`

    - Byline ends in "Detroit News * Bureau", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*detroit\s*news\s*.*\s*bureau$' )`   

In [ ]:
# filter queryset to just locally created Detroit News (TDN) articles.
# imports
from context_text.models import Article
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase
from context_text.collectors.newsbank.newspapers.DTNB import DTNB

# declare variables - Detroit News
do_apply_tag = False
tag_to_apply = None
tdn_local_news_sections = []
tdn_newspaper = None
tdn_article_qs = None
article_count = -1

# declare variables - filtering
include_opinion_columns = True
tags_in_list = []
tags_not_in_list = []
filter_out_prelim_tags = False
random_count = -1

# declare variables - make list of article IDs from QS.
article_id_list = []
article_counter = -1
current_article = None

# ==> configure

# configure - size of random sample we want
#random_count = 60

# configure - also, apply tag?
do_apply_tag = False
tag_to_apply = ContextTextBase.TAG_LOCAL_HARD_NEWS

# set up "local, regional and state news" sections
tdn_local_news_sections = DTNB.LOCAL_NEWS_SECTION_NAME_LIST

# Detroit News
# get newspaper instance for TDN.
tdn_newspaper = Newspaper.objects.get( id = DTNB.NEWSPAPER_ID )

# start with all articles
#tdn_article_qs = Article.objects.all()

# ==> filter to newspaper, local news section list, and in-house reporters.

# ----> manually

# now, need to find local news articles to test on.
#tdn_article_qs = tdn_article_qs.filter( newspaper = tdn_newspaper )

# only the locally implemented sections
#tdn_article_qs = tdn_article_qs.filter( section__in = tdn_local_news_sections )

# and, with an in-house author
#tdn_article_qs = tdn_article_qs.filter( DTNB.Q_IN_HOUSE_AUTHOR )

#print( "manual filter count: {}".format( tdn_article_qs.count() ) )

# ----> using Article.filter_articles()
tdn_article_qs = Article.filter_articles( qs_IN = tdn_article_qs,
                                          newspaper = tdn_newspaper,
                                          section_name_list = tdn_local_news_sections,
                                          custom_article_q = DTNB.Q_IN_HOUSE_AUTHOR )

print( "Article.filter_articles count: {}".format( tdn_article_qs.count() ) )

# and include opinion columns?
if ( include_opinion_columns == False ):
    
    # do not include columns
    tdn_article_qs = tdn_article_qs.exclude( author_string__in = DTNB.COLUMNIST_NAME_LIST )
    
#-- END check to see if we include columns. --#

'''
# filter to newspaper, section list, and in-house reporters.
tdn_article_qs = Article.filter_articles( qs_IN = tdn_article_qs,
                                          start_date = "2009-12-01",
                                          end_date = "2009-12-31",
                                          newspaper = tdn_newspaper,
                                          section_name_list = tdn_local_news_sections,
                                          custom_article_q = DTNB.Q_IN_HOUSE_AUTHOR )
'''

# how many is that?
article_count = tdn_article_qs.count()

print( "Article count before filtering on tags: " + str( article_count ) )

# ==> tags

# tags to exclude
#tags_not_in_list = [ "prelim_reliability", "prelim_network" ]
#tags_not_in_list = [ "minnesota1-20160328", "minnesota2-20160328", ]

# for later - exclude articles already coded.
#tags_not_in_list = [ OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME ]

tags_not_in_list = None
if ( ( tags_not_in_list is not None ) and ( len( tags_not_in_list ) > 0 ) ):

    # exclude those in a list
    print( "filtering out articles with tags: " + str( tags_not_in_list ) )
    tdn_article_qs = tdn_article_qs.exclude( tags__name__in = tags_not_in_list )

#-- END check to see if we have a specific list of tags we want to exclude --#

# include only those with certain tags.
#tags_in_list = [ "prelim_unit_test_001", "prelim_unit_test_002", "prelim_unit_test_003", "prelim_unit_test_004", "prelim_unit_test_005", "prelim_unit_test_006", "prelim_unit_test_007" ]
#tags_in_list = [ "tdn_month", ]
tags_in_list = None
if ( ( tags_in_list is not None ) and ( len( tags_in_list ) > 0 ) ):

    # filter
    print( "filtering to just articles with tags: " + str( tags_in_list ) )
    tdn_article_qs = tdn_article_qs.filter( tags__name__in = tags_in_list )
    
#-- END check to see if we have a specific list of tags we want to include --#

# filter out "*prelim*" tags?
#filter_out_prelim_tags = True
if ( filter_out_prelim_tags == True ):

    # ifilter out all articles with any tag whose name contains "prelim".
    print( "filtering out articles with tags that contain \"prelim\"" )
    tdn_article_qs = tdn_article_qs.exclude( tags__name__icontains = "prelim" )
    
#-- END check to see if we filter out "prelim_*" tags --#

# how many is that?
article_count = tdn_article_qs.count()

print( "Article count after tag filtering: " + str( article_count ) )

# do we want a random sample?
if ( random_count > 0 ):

    # to get random, order them by "?", then use slicing to retrieve requested
    #     number.
    tdn_article_qs = tdn_article_qs.order_by( "?" )[ : random_count ]
    
#-- END check to see if we want random sample --#

# this is a nice algorithm, also:
# - http://www.titov.net/2005/09/21/do-not-use-order-by-rand-or-how-to-get-random-rows-from-table/

# make ID list, tag articles if configured to.
article_id_list = []
article_counter = 0
for current_article in tdn_article_qs:

    # increment article_counter
    article_counter += 1

    # add IDs to article_id_list
    article_id_list.append( str( current_article.id ) )
    
    # apply a tag while we are at it?
    if ( ( do_apply_tag == True ) and ( tag_to_apply is not None ) and ( tag_to_apply != "" ) ):
    
        # yes, please.  Add tag.
        current_article.tags.add( tag_to_apply )
        
    #-- END check to see if we apply tag. --#

    # output the tags.
    if ( debug_flag == True ):
        print( "- Tags for article " + str( current_article.id ) + " : " + str( current_article.tags.all() ) )
    #-- END DEBUG --#

#-- END loop over articles --#

# output the list.
print( "tdn_article_qs count: {}".format( tdn_article_qs.count() ) )
print( "Found " + str( article_counter ) + " articles ( " + str( article_count ) + " )." )
if ( debug_flag == True ):
    print( "List of " + str( len( article_id_list ) ) + " local TDN staff article IDs: " + ", ".join( article_id_list ) )
#-- END DEBUG --#


# Code Articles

- Back to [Table of Contents](#Table-of-Contente)

Retrieve just publications that are tagged as being local hard news and that also are not tagged as having been coded by OpenCalaisV2.

In [8]:
# declare variables

# declare variables - article filter parameters
start_pub_date = None # should be datetime instance
end_pub_date = None # should be datetime instance
tags_in_list = []
tags_not_in_list = []
paper_id_in_list = []
section_list = []
article_id_in_list = []
params = {}

# declare variables - processing
do_i_print_updates = True
my_article_coding = None
article_qs = None
article_count = -1
coding_status = ""
limit_to = -1
do_coding = True

# declare variables - results
success_count = -1
success_list = None
got_errors = False
error_count = -1
error_dictionary = None
error_article_id = -1
error_status_list = None
error_status = ""
error_status_counter = -1

# first, get a list of articles to code.

# ! Set param values.

# ==> start and end dates
#start_pub_date = "2009-12-06"
#end_pub_date = "2009-12-12"

# ==> tagged articles

# Examples:
#tag_in_list = "prelim_reliability"
#tag_in_list = "prelim_network"
#tag_in_list = "prelim_unit_test_007"
#tag_in_list = [ "prelim_reliability", "prelim_network" ]
#tag_in_list = [ "prelim_reliability_test" ] # 60 articles - Grand Rapids only.
#tag_in_list = [ "prelim_reliability_combined" ] # 87 articles, Grand Rapids and Detroit.
#tag_in_list = [ "prelim_training_001" ]
#tag_in_list = [ "grp_month" ]

# ----> include articles when these tags are present.
#tags_in_list = None
tags_in_list = []
tags_in_list.append( ContextTextBase.TAG_LOCAL_HARD_NEWS )

# ---> exclude articles when these tags are present.
#tags_not_in_list = None
tags_not_in_list = []
tags_not_in_list.append( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )

# ==> IDs of newspapers to include.
#paper_id_in_list = "1"

# ==> names of sections to include.
#section_list = "Lakeshore,Front Page,City and Region,Business"

# ==> just limit to specific articles by ID.
article_id_in_list = []
#article_id_in_list = [ 360962 ]
#article_id_in_list = [ 28598 ]
#article_id_in_list = [ 21653, 21756 ]
#article_id_in_list = [ 90948 ]
#article_id_in_list = [ 21627, 21609, 21579 ]
#article_id_in_list = [ 48778 ]
#article_id_in_list = [ 6065 ]
#article_id_in_list = [ 221858 ]
#article_id_in_list = [ 23804, 22630 ]
#article_id_in_list = [ 23804 ]

# debugging exception
#article_id_in_list.append( 402670 )
#article_id_in_list.append( 408735 )

# filter parameters
params[ ArticleCoding.PARAM_START_DATE ] = start_pub_date
params[ ArticleCoding.PARAM_END_DATE ] = end_pub_date
params[ ArticleCoding.PARAM_TAGS_IN_LIST ] = tags_in_list
params[ ArticleCoding.PARAM_TAGS_NOT_IN_LIST ] = tags_not_in_list
params[ ArticleCoding.PARAM_PUBLICATION_LIST ] = paper_id_in_list
params[ ArticleCoding.PARAM_SECTION_LIST ] = section_list
params[ ArticleCoding.PARAM_ARTICLE_ID_LIST ] = article_id_in_list

# set coder you want to use.

# OpenCalais REST API v.2
params[ ArticleCoding.PARAM_CODER_TYPE ] = ArticleCoding.ARTICLE_CODING_IMPL_OPEN_CALAIS_API_V2

# get instance of ArticleCoding
my_article_coding = ArticleCoding()
my_article_coding.do_print_updates = do_i_print_updates

# to adjust timing, you need to update the ArticleCoder class for your
#     coder.  That overrides the value set here (so we respect limits
#     if they are coded into a particular coder):
my_article_coding.rate_limit_in_seconds = 3

# set params
my_article_coding.store_parameters( params )

print( "Query Parameters: {}".format( params ) )

# create query set - ArticleCoding does the filtering for you.
article_qs = my_article_coding.create_article_query_set()

print( "After my_article_coding.create_article_query_set(), count: {}".format( article_qs.count() ) )
if ( article_qs._result_cache is None ):
    
    print( "article_qs evaluated: NO ( {} )".format( article_qs._result_cache ) )
    
else:
    
    print( "article_qs evaluated: YES" )

#-- END check to see if _result_cache --#

# order by pub_date DESC, so we do most recent first.
article_qs = article_qs.order_by( "-pub_date" )

# limit for an initial test?
limit_to = 4990
if ( ( limit_to is not None ) and ( isinstance( limit_to, int ) == True ) and ( limit_to > 0 ) ):

    # yes.
    article_qs = article_qs[ : limit_to ]

#-- END check to see if limit --#

# get article count
if ( isinstance( article_qs, list ) == True ):

    # list - call len()
    article_list = article_qs
    article_count = len( article_list )
    
else:

    # not a list - call count()
    article_count = article_qs.count()
    
#-- END figure out how to get count --#

print( "Matching article count: " + str( article_count ) )

# Do coding?
if ( do_coding == True ):

    print( "do_coding == True - it's on!" )

    # yes - make sure we have at least one article:
    if ( article_count > 0 ):

        # invoke the code_article_data( self, query_set_IN ) method.
        coding_status = my_article_coding.code_article_data( article_qs )
    
        # output status
        print( "\n\n==============================\n\nCoding status: \"" + coding_status + "\"" )
        
        # get success count
        success_count = my_article_coding.get_success_count()
        print( "\n\n====> Count of articles successfully processed: " + str( success_count ) )    
        
        # if successes, list out IDs.
        if ( success_count > 0 ):
        
            # there were successes.
            success_list = my_article_coding.get_success_list()
            print( "- list of successfully processed articles: " + str( success_list ) )
        
        #-- END check to see if successes. --#
        
        # got errors?
        got_errors = my_article_coding.has_errors()
        if ( got_errors == True ):
        
            # get error dictionary
            error_dictionary = my_article_coding.get_error_dictionary()
            
            # get error count
            error_count = len( error_dictionary )
            print( "\n\n====> Count of articles with errors: " + str( error_count ) )
            
            # loop...
            for error_article_id, error_status_list in six.iteritems( error_dictionary ):
            
                # output errors for this article.
                print( "- errors for article ID " + str( error_article_id ) + ":" )
                
                # loop over status messages.
                error_status_counter = 0
                for error_status in error_status_list:
                
                    # increment status
                    error_status_counter += 1

                    # print status
                    print( "----> status #" + str( error_status_counter ) + ": " + error_status )
                    
                #-- END loop over status messages. --#
            
            #-- END loop over articles. --#
   
        #-- END check to see if errors --#
    
    #-- END check to see if article count. --#
    
else:
    
    # output matching article count.
    print( "do_coding == False, so dry run" )
    
#-- END check to see if we do_coding --#


Query Parameters: {'start_date': None, 'end_date': None, 'tags_in_list_IN': ['local_hard_news'], 'tags_not_in_list_IN': ['coded-OpenCalaisV2ArticleCoder'], 'publications': [], 'section_list': [], 'article_id_list': [], 'coder_type': 'open_calais_api_v2'}
After my_article_coding.create_article_query_set(), count: 38325
article_qs evaluated: NO ( None )
Matching article count: 4990
do_coding == True - it's on!


==> article 1: 69370 - Rape suspect spurns plea deal - Life sentence possible for alleged school attack


==> article 2: 69368 - New budget chief to tackle Michigan's fiscal crisis - State offers 'great opportunity' for reform, Utah official says


==> article 3: 69385 - Lions Club balks at fee for parade


==> article 4: 69406 - Kentwood Pharmacy split, sold at auction - Creditor gets most; wife of Eastgate former owner buys piece


==> article 5: 69410 - Slaying baffles victim's family - Suspect arraigned, but motive still unclear


==> article 6: 69413 - Work at home - Law all



==> article 48: 68955 - Changes planned for Chicago Drive - Two left turns from 32nd Avenue may be redesigned to help trucks


==> article 49: 68966 - Owners of 40 homes wonder: Tax error in our favor? - Grand Rapids Township neighborhood may have been overbilled $32,000


==> article 50: 68968 - You can consider it doctor shopping - Metro Health plans session for matchmaking patients with ob-gyns


==> article 51: 68903 - Making a difference - Volunteers feed on energy of helping the less-fortunate


==> article 52: 68929 - Suspect search ends in suicide - Man wanted in GR slaying stabs self


==> article 53: 68907 - Workers OK freeze, with a catch - Other Grandville unions will need to follow suit


==> article 54: 68915 - The earlier, the better - Shoppers decide not to wait until friday to get their deals


==> article 55: 68963 - Welcoming the world - European connection keeps antiques mall active


==> article 56: 68959 - Homeowners asked to pitch in for sidewalks - Special ass



==> article 98: 68605 - Quick action praised at accident scene - Brother of teen who died kept watch over cousins


==> article 99: 68604 - Suspect says cat he killed was feral - Drowning has brought animal-cruelty charges


==> article 100: 68615 - Foster mom declines 2nd plea deal - She is accused of shoving girl hard enough to cause child's death


==> article 101: 68670 - GRPS reviews popular program - If interest remains high, Montessori school could add grade levels next year


==> article 102: 68645 - Bus driver pay not taking city for a ride, website shows


==> article 103: 68527 - It is written ... - 115 letters reveal deep support for convicted sex offender


==> article 104: 68551 - Area entrepreneur has Titanic ideas


==> article 105: 68556 - Jamestown water-sewer costs an up-and-down proposition - Hook-up fees reduced, meter service added for 800 sewer users


==> article 106: 68596 - Don't break the bank on ring for fiancee


==> article 107: 68563 - Township to limit



==> article 144: 68135 - Feds say drugs sold in Cadillac - Kentwood Pharmacy executive accused of supplying dealer


==> article 145: 68180 - Deer killed near U.P. border doesn't have wasting disease - Positive test would have meant destroying herd, ban on baiting


==> article 146: 68167 - Thanksgiving rising - Price of holiday meal is up this year, but not too much


==> article 147: 68181 - Anti-discrimination law could include exceptions


==> article 148: 68123 - Endangered trumpeter swan shot - Bird believed to have been wounded in flight over Roscommon County


==> article 149: 68129 - Study focuses on climate change - Grand Rapids is one of inaugural cities receiving database tool


==> article 150: 68155 - Teachers object to online program - GRPS blended learning hurts students, they say


==> article 151: 68152 - MSU medical students skate to help patient - Hockey game to benefit classmate's brother, who is battling brain cancer


==> article 152: 68173 - Nothing beats good



==> article 192: 68018 - Is free delivery of medicine just what the doctor ordered? - Customer calls program 'a godsend;' grocers says its good business


==> article 193: 67856 - Voucher hopefuls flood phone lines - Thousands apply for a spot on housing subsidy wait list


==> article 194: 67886 - Someone has something to smile about, but who? - Mystery campaign spreads happy signs throughout area


==> article 195: 67878 - Fundraising failure teaches college-level lesson - Benevolent competition sees GVSU beaten by $37,000


==> article 196: 67821 - Leadership search to start - Superintendent head-hunters to be interviewed next month


==> article 197: 67839 - Court resolves giraffe prank - Five Calvin students will take part in diversion program


==> article 198: 67847 - GRCC salary study delayed - Time needed to compare with other institutions


==> article 199: 67828 - Publisher addresses changing media landscape - Dan Gaydou: Technological pace is accelerating


==> article 20

...



==> article 4780: 28501 - Not in school? You'll miss the fun - Parties are one incentive to raise attendance


==> article 4781: 28511 - City manager unveils five-year tax hike - Grand Rapids commissioners could put income tax issue on may 4 ballot


==> article 4782: 28508 - In Haiti, operating on hope - West Michigan physicians donate their talents, and their own blood, to quake-ravaged people


==> article 4783: 28381 - Task force takes on teen drinking - Study finds alcohol easy to obtain; Grant helps fund education efforts


==> article 4784: 28388 - For food and fuel - Farm-based ethanol processor in development


==> article 4785: 28407 - Continue with job or start small business? - Which is best financial move for couple starting family?


==> article 4786: 28397 - School: Part of a complete breakfast - Godwin Heights joins trend of providing morning meal to all students for free


==> article 4787: 28375 - Friends mourn devoted Christian - Grand Rapids man, 25, died in rollo



==> article 4825: 28097 - When worlds collide - 'Twilight' fans find religious themes lurk in underworld characters


==> article 4826: 28062 - Traffic stop nets drugs, cash, gun - More evidence recovered at driver's residence


==> article 4827: 28140 - Speaker shares faith journey - Boston College professor converted to Catholicism from Protestantism


==> article 4828: 28066 - Governor gets birthday greetings from workers


==> article 4829: 28036 - County deer free of CWD so far - Testing will continue for a third year


==> article 4830: 28046 - Toyota still draws interest - Potential customers flock to annual auto show


==> article 4831: 27984 - C'mon, it doesn't get much cooler than this - Embracing our cold months could boost Michigan's economy and reputation


==> article 4832: 28009 - District to delay decision on closing buildings - Board scheduled to decide March 1


==> article 4833: 27969 - Film studio attracts attention from Europe - Hangar42 Studios could provide as 



==> article 4872: 27770 - Minorities sought for bone-marrow drive - Police, firefighters host event to enlist people to join national registry


==> article 4873: 27847 - Paintball business wants to expand - Grandville worries about complaints from neighbors


==> article 4874: 27814 - Kentwood puts marijuana on hold - Moratorium gives city time to form regulations for medicinal uses


==> article 4875: 27835 - Amway exec headlines Inforum dinner - Candace Matthews puts mentors at the top of her business list


==> article 4876: 27834 - GR joins tax hike bandwagon - Voters may be asked to OK 15.4 percent income tax increase in May


==> article 4877: 27870 - Homeless center gets nod - Neighbors voice concerns about facility's impact


==> article 4878: 27782 - Watershed council to conduct $97,000 study of river acreage


==> article 4879: 27696 - Parents fume over closures - West Siders complain they may be taking the biggest hit


==> article 4880: 27690 - Steelcase lays-off workers



==> article 4910: 27494 - Reinventing yourself - If it's time to retool the resume, here is what experts recommend


==> article 4911: 27359 - A split decision - Five years after Zeeland athletics divided in two, questions linger


==> article 4912: 27400 - Kazen returns home as Albion's starting forward - Former West Ottawa and Saugatuck standout reaches new career high


==> article 4913: 27459 - $5 bet propels Giant of Giants to take place in history - Mary Edmond takes top honor at 28th annual banquet


==> article 4914: 27399 - Students get leg up on fitness in Zeeland - Educator creates yoga program using tunes to teach moves


==> article 4915: 27344 - Hamilton's Koning commits to GVSU - Two-year captain will likely play linebacker and redshirt for Lakers


==> article 4916: 27342 - Heads nod as GR mayor calls for partners - Politicians agree on need for more governmental cooperation


==> article 4917: 27429 - West Ottawa's new track coach ready for season - Kingma looking to



==> article 4956: 27184 - Thanks to ABC, we're all invited to 'Bachelor' nuptials - Grand Rapids native Molly Malaney's wedding will air March 8


==> article 4957: 27210 - Holiday's over: State jobless rate jumps


==> article 4958: 27241 - Dealer appeals to keep GM vehicles - Good says it was unfairly targeted to lose franchise


==> article 4959: 27259 - Steelcase exec's pay restored - Office-furniture market unchanged, official says


==> article 4960: 27195 - Gentex posts $30 million fourth-quarter profit - Zeeland company has plans to hire 150 more people


==> article 4961: 27120 - Mother takes girl across border - Greenville father fights to bring daughter back from Ecuador


==> article 4962: 27087 - Airport lands AirTran, hopes price war takes off


==> article 4963: 27078 - Who is a Hollander? - Council to consider tightening its definition of a city resident


==> article 4964: 27077 - Economist predicts 'precarious' recovery - Speaker at Home and Building Association of 

- 2019.07.31 - 5000 - started: execution queued 22:42:01 2019-07-31 --> executed in 3h 47m 55s, finished 02:29:55 2019-08-01
- 2019.08.03 - 4990 - started: execution queued 00:38:05 2019-08-03 --> 

In [10]:
print( "time per transaction: {}".format( my_article_coding.rate_limit_in_seconds ) )
my_article_coding.rate_limit_in_seconds = 3
print( "time per transaction: {}".format( my_article_coding.rate_limit_in_seconds ) )

time per transaction: 2
time per transaction: 3


## Optional Validation

- Back to [Table of Contents](#Table-of-Contents)

In [11]:
# get automated coder
automated_coder_user = ArticleCoder.get_automated_coding_user()

print( "{} - Loaded automated user: {}, id = {}".format( datetime.datetime.now(), automated_coder_user, automated_coder_user.id ) )

2019-08-03 11:53:27.728417 - Loaded automated user: automated, id = 2


### Validate success publications

- Back to [Table of Contents](#Table-of-Contents)

Loop over all successful records and verify:

- that they have the OpenCalais coded-by-me tag (`OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME`).
- that they have an ArticleData for automated coding user.
- that it isn't all just 0 sources.  Perhaps, collect and average source and subject counts.

In [12]:
# declare variables
success_count = -1
success_list = None
article_id = None
has_coded_tag = None
has_coded_tag_counter = None
has_article_data_counter = None
article_instance = None

# declare variables - tag validation
tag_name_list = None
coded_by_tag_name = None
has_coded_by_tag = None

# declare variables - ArticleData validation
article_id_to_data_map = None
article_data_qs = None
article_data_count = None
article_data_instance = None
article_data_id = None
automated_coder_type = None
article_data_map = None
article_author_qs = None
author_count = None
article_subject_qs = None
subject_qs = None
subject_count = None
source_qs = None
source_count = None
has_data_count = None
has_people_count = None
has_subjects_count = None
has_sources_count = None

# validation

# init
coded_by_tag_name = OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME
#automated_coder_user = ArticleCoder.get_automated_coding_user()
automated_coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION
article_id_to_data_map = {}

# get success count
success_count = my_article_coding.get_success_count()
print( "====> Count of articles successfully processed: " + str( success_count ) )    

# if successes, list out IDs.
if ( success_count > 0 ):

    # there were successes.
    success_list = my_article_coding.get_success_list()
    #print( "- list of successfully processed articles: " + str( success_list ) )
    
    # loop over success articles
    has_coded_tag_counter = 0
    has_article_data_counter = 0
    has_data_count = 0
    has_people_count = 0
    has_subjects_count = 0
    has_sources_count = 0
    for article_id in success_list:
        
        # load article
        article_instance = Article.objects.get( pk = article_id )
        
        # get tag name list
        tag_name_list = article_instance.tags.names()
        
        # is coded-by tag name present?
        if ( coded_by_tag_name in tag_name_list ):
            
            # it is there, as it should be.
            has_coded_by_tag =  True
            has_coded_tag_counter += 1
            
        else:
            
            # not there.  Error.
            has_coded_by_tag = False
            print( "ERROR in article {}: coded-by tag ( {} ) not in tag list: {}".format( article_id, coded_by_tag_name, tag_name_list ) )
        
        #-- END check for coded-by tag name in tag list. --#
        
        # is there an ArticleData instance by automated coder for OpenCalais V.2?
        article_data_qs = article_instance.article_data_set.filter( coder = automated_coder_user )
        article_data_qs = article_data_qs.filter( coder_type = automated_coder_type )
        article_data_count = article_data_qs.count()
        if ( article_data_count == 1 ):
            
            # got one.  Increment counter.
            has_article_data_counter += 1
            
            # TODO - check how many sources, subjects.
            article_data_instance = article_data_qs.get()
            article_data_id = article_data_instance.id
            
            # create article data map
            article_data_map = {}
            article_data_map[ "article_id" ] = article_id
            article_data_map[ "article_instance" ] = article_instance
            article_data_map[ "article_data_instance" ] = article_data_instance
            article_data_map[ "article_data_id" ] = article_data_id
            
            # get count of authors
            article_author_qs = article_data_instance.article_author_set.all()
            author_count = article_author_qs.count()
            article_data_map[ "author_count" ] = author_count
            
            # get count of subjects
            article_subject_qs = article_data_instance.article_subject_set.all()
            article_subject_total_count = article_subject_qs.count()
            article_data_map[ "article_subject_total_count" ] = article_subject_total_count
            if ( article_subject_total_count > 0 ):
                
                has_people_count += 1
                
            #-- END check to see if any people found at all --#
            
            # just subjects
            subject_qs = article_subject_qs.filter( subject_type = Article_Subject.SUBJECT_TYPE_MENTIONED )
            subject_count = subject_qs.count()
            article_data_map[ "subject_count" ] = subject_count
            if ( subject_count > 0 ):
                
                has_subjects_count += 1
                
            #-- END check to see if any subjects found --#
            
            # get count of sources
            source_qs = article_subject_qs.filter( subject_type = Article_Subject.SUBJECT_TYPE_QUOTED )
            source_count = source_qs.count()
            article_data_map[ "source_count" ] = source_count
            if ( source_count > 0 ):
                
                has_sources_count += 1
                
            #-- END check to see if any sources found --#
            
            # store information for article.
            article_id_to_data_map[ article_id ] = article_data_map
            
            if ( ( author_count == 0 ) and ( article_subject_total_count == 0 ) ):
                
                print( "No authors or sources in article {}".format( article_id ) )
                
            else:
                
                # increment populated data count
                has_data_count += 1
                
            #-- END sanity check for empty data (won't be zero, shouldn't be many) --#
            
        elif ( article_data_count > 1 ):
            
            # more than one?
            print( "ERROR in article {}: more than one ArticleData instance ( {} ) for automated coder ( {} ), coder type: {}.".format( article_id, article_data_count, automated_coder_user, automated_coder_type ) )
            
        else:
            
            # error - no ArticleData.
            print( "ERROR in article {}: no ArticleData instances for automated coder ( {} ), coder type: {}.".format( article_id, automated_coder_user, automated_coder_type ) )
            
        #-- END check to see if ArticleData by automated coder, Open Calais v.2 --#
        
    #-- END loop over IDs of sucessfully processed articles. --#

#-- END check to see if successes. --#
        
print( "- Tagged article count: {}".format( has_coded_tag_counter ) )
print( "- Correct ArticleData count: {}".format( has_article_data_counter ) )
print( "- Has data count: {}".format( has_data_count ) )
print( "- Has people count: {}".format( has_people_count ) )
print( "- Has subjects count: {}".format( has_subjects_count ) )
print( "- Has sources count: {}".format( has_sources_count ) )

====> Count of articles successfully processed: 4925
- Tagged article count: 4925
- Correct ArticleData count: 4925
- Has data count: 4925
- Has people count: 4813
- Has subjects count: 3313
- Has sources count: 4582


### Validate error publications

- Back to [Table of Contents](#Table-of-Contents)

Loop over all error records and verify:

- that they do not have the OpenCalais coded-by-me tag (`OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME`).
- check on the status of their ArticleData.  Do they have any?  If so, what to do?

In [13]:
# declare variables
got_errors = None
error_dictionary = None
error_count = None
error_article_id = None
error_status_list = None
error_status_counter = None
article_instance = None
tag_name_list = None
coded_by_tag_name = None
has_coded_by_tag = None

# declare variables - ArticleData validation
error_article_id_to_data_map = None
article_data_qs = None
article_data_count = None
article_data_instance = None
article_data_id = None
automated_coder_type = None
article_data_map = None
article_author_qs = None
author_count = None
article_subject_qs = None
subject_qs = None
subject_count = None
source_qs = None
source_count = None
has_data_count = None
has_people_count = None
has_subjects_count = None
has_sources_count = None

# init
coded_by_tag_name = OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME
#automated_coder_user = ArticleCoder.get_automated_coding_user()
automated_coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION
error_article_id_to_data_map = {}

# got errors?
got_errors = my_article_coding.has_errors()
if ( got_errors == True ):

    # get error dictionary
    error_dictionary = my_article_coding.get_error_dictionary()

    # get error count
    error_count = len( error_dictionary )
    print( "====> Count of articles with errors: " + str( error_count ) )

    # loop...
    has_coded_tag_counter = 0
    has_article_data_counter = 0
    has_data_count = 0
    has_people_count = 0
    has_subjects_count = 0
    has_sources_count = 0
    for error_article_id, error_status_list in six.iteritems( error_dictionary ):

        print( "\nError article ID: {}".format( error_article_id ) )

        # output errors for this article.
        print( "- errors for article ID {}:".format( error_article_id ) )

        # loop over status messages.
        error_status_counter = 0
        for error_status in error_status_list:

            # increment status
            error_status_counter += 1

            # print status
            print( "----> status #" + str( error_status_counter ) + ": " + error_status )

        #-- END loop over status messages. --#

        # load article
        article_instance = Article.objects.get( pk = error_article_id )
        
        # get tag name list
        tag_name_list = article_instance.tags.names()
        
        # is coded-by tag name present?
        if ( coded_by_tag_name in tag_name_list ):
            
            # it is there, as it should be.
            has_coded_by_tag =  True
            has_coded_tag_counter += 1
            
        else:
            
            # not there.  Error.
            has_coded_by_tag = False
            #print( "ERROR in article {}: coded-by tag ( {} ) not in tag list: {}".format( error_article_id, coded_by_tag_name, tag_name_list ) )
        
        #-- END check for coded-by tag name in tag list. --#
        
        # is there an ArticleData instance by automated coder for OpenCalais V.2?
        article_data_qs = article_instance.article_data_set.filter( coder = automated_coder_user )
        article_data_qs = article_data_qs.filter( coder_type = automated_coder_type )
        article_data_count = article_data_qs.count()
        if ( article_data_count == 1 ):
            
            # got one.  Increment counter.
            has_article_data_counter += 1
            
            # TODO - check how many sources, subjects.
            article_data_instance = article_data_qs.get()
            article_data_id = article_data_instance.id
            
            # create article data map
            article_data_map = {}
            article_data_map[ "article_id" ] = error_article_id
            article_data_map[ "article_instance" ] = article_instance
            article_data_map[ "article_data_instance" ] = article_data_instance
            article_data_map[ "article_data_id" ] = article_data_id
            
            # get count of authors
            article_author_qs = article_data_instance.article_author_set.all()
            author_count = article_author_qs.count()
            article_data_map[ "author_count" ] = author_count
            
            # get count of subjects
            article_subject_qs = article_data_instance.article_subject_set.all()
            article_subject_total_count = article_subject_qs.count()
            article_data_map[ "article_subject_total_count" ] = article_subject_total_count
            if ( article_subject_total_count > 0 ):
                
                has_people_count += 1
                
            #-- END check to see if any people found at all --#
            
            # just subjects
            subject_qs = article_subject_qs.filter( subject_type = Article_Subject.SUBJECT_TYPE_MENTIONED )
            subject_count = subject_qs.count()
            article_data_map[ "subject_count" ] = subject_count
            if ( subject_count > 0 ):
                
                has_subjects_count += 1
                
            #-- END check to see if any subjects found --#
            
            # get count of sources
            source_qs = article_subject_qs.filter( subject_type = Article_Subject.SUBJECT_TYPE_QUOTED )
            source_count = source_qs.count()
            article_data_map[ "source_count" ] = source_count
            if ( source_count > 0 ):
                
                has_sources_count += 1
                
            #-- END check to see if any sources found --#
            
            # store information for article.
            error_article_id_to_data_map[ error_article_id ] = article_data_map
            
            if ( ( author_count == 0 ) and ( article_subject_total_count == 0 ) ):
                
                pass
                #print( "- No authors or sources in article {}".format( error_article_id ) )
                
            else:
                
                # increment populated data count
                has_data_count += 1
                print( "- Found data in article {}: person = {}; subject = {}; source = {}".format( error_article_id, article_subject_total_count, subject_count, source_count ) )
                
            #-- END sanity check for empty data (won't be zero, shouldn't be many) --#
            
        elif ( article_data_count > 1 ):
            
            # more than one?
            print( "ERROR in article {}: more than one ArticleData instance ( {} ) for automated coder ( {} ), coder type: {}.".format( error_article_id, article_data_count, automated_coder_user, automated_coder_type ) )
            
        else:
            
            # no ArticleData.
            pass
            
        #-- END check to see if ArticleData by automated coder, Open Calais v.2 --#

    #-- END loop over articles. --#

    print( "- Tagged article count: {}".format( has_coded_tag_counter ) )
    print( "- Correct ArticleData count: {}".format( has_article_data_counter ) )
    print( "- Has data count: {}".format( has_data_count ) )
    print( "- Has people count: {}".format( has_people_count ) )
    print( "- Has subjects count: {}".format( has_subjects_count ) )
    print( "- Has sources count: {}".format( has_sources_count ) )

else:

    print( "NO ERRORS!  YAY!" )
    
#-- END check to see if errors --#


====> Count of articles with errors: 65

Error article ID: 67680
- errors for article ID 67680:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 67680 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.

Error article ID: 67574
- errors for article ID 67574:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 67574 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.

Error article ID: 66214
- errors for article ID 66214:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 66214 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.

Error article ID: 66243
- errors for article ID 66243:
----> status #1: Error: ValueError parsing OpenCa


Error article ID: 41859
- errors for article ID 41859:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 41859 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.

Error article ID: 41741
- errors for article ID 41741:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 41741 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.

Error article ID: 40430
- errors for article ID 40430:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 40430 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.

Error article ID: 40338
- errors for article ID 40338:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 40338 - raw respon


Error article ID: 27373
- errors for article ID 27373:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 27373 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.

Error article ID: 27356
- errors for article ID 27356:
----> status #1: Error: ValueError parsing OpenCalais JSON for Article 27356 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.
- Tagged article count: 0
- Correct ArticleData count: 65
- Has data count: 0
- Has people count: 0
- Has subjects count: 0
- Has sources count: 0


NOTE: Looks like publications where there is an OpenCalais network error are not getting the Coded tag applied, so they will remain in the pool to be re-coded in subsequent runs.

In [14]:
# get list of error IDs from map.
if ( error_dictionary is not None ):

    error_article_id_list = list( six.viewkeys( error_dictionary ) )
    print( "IDs of articles with errors: {}".format( error_article_id_list ) )
    
else:

    print( "STILL NO ERRORS!  YAY!" )
    
#-- END check to see if None --#

IDs of articles with errors: [67680, 67574, 66214, 66243, 66130, 66000, 64658, 64654, 63244, 63220, 61836, 61842, 61745, 61667, 60344, 60369, 58959, 58893, 57742, 57506, 6496, 6402, 6343, 5212, 5169, 5144, 4243, 4059, 3980, 43422, 43406, 43372, 41926, 41859, 41741, 40430, 40338, 40358, 38901, 38869, 38954, 37950, 37925, 37946, 37968, 36505, 36449, 36499, 36508, 34976, 35055, 34977, 35009, 33459, 33452, 31958, 31920, 30451, 30339, 29002, 28887, 28994, 28940, 27373, 27356]


# TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- start loading data from XML
- move data from Article_Data into context.
- make the network data creator work against context, then generalize it for tie and node types.
- think how we specify which class to use for author strings - needs to be speced to an interface, but not just a newsbank one - so, abstraction here should be higher up - in shared?

DONE:

- // Save log of coding first 4990 of next round of data.
- // for next round of coding, sort on publication date, descending, so we fill in the year before and after the layoffs first.
- // adjust django logging to output DEBUG, then test Article.filter_articles() to see where QuerySet is evaluated (DISTINCT check?).
